In [14]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [23]:
def drop_sers_with_nans(df, from_axis='rows'):
    
    if df.isnull().values.any()==True:
        print(f'Dropped {from_axis} with NaNs!')

        if from_axis=='rows':
            dff = df.dropna(axis=0, inplace=False)
            dff.reset_index(drop=True, inplace=True)
            print(f'Num rows before: {df.shape[0]}')
            print(f'Num rows after: {dff.shape[0]}')
            
        elif from_axis=='cols':
            dff = df.dropna(axis=1, inplace=False)
            print(f'Num cols before: {(df.shape[1])}')
            print(f'Num cols after: {(dff.shape[1])}')
            
        return dff
    
    else:
        print(f'No NaNs! :)')
        return df

In [24]:
timestamp = '20240928-145111'

fpath = os.path.join(DATA_DIR,'processed',f'formatted_state_fips_{timestamp}.csv')
df_fips = pd.read_csv(fpath)

# fpath = os.path.join(DATA_DIR,'processed',f'formatted_COVIDhub-ensemble_{timestamp}.csv')
# df_hub = pd.read_csv(fpath)

# fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{timestamp}.csv')
# df_nyt = pd.read_csv(fpath)

fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_{timestamp}.csv')
df_acs = pd.read_csv(fpath)

display(df_fips.head())
display(df_acs)

,State,Postal,Fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


,Fips,POP_x2,POP_M,POP_F,POP_A0004,POP_A0509,POP_A1014,POP_A1517,POP_A1819,POP_A20,...,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,2013,3409,2014,1395,122,103,151,106,52,37,...,1113,835,3,57,72,41,35,0,0,70
1,2016,5251,2995,2256,162,215,148,110,154,55,...,1456,417,70,242,275,154,79,180,12,27
2,2020,292545,149648,142897,20218,20836,18642,11308,7022,3885,...,118055,56649,16185,6372,12729,6852,4719,6892,2642,5015
3,2050,18514,9724,8790,1902,1923,1696,1028,612,261,...,5992,4826,125,305,228,56,65,150,9,228
4,2060,849,480,369,67,41,28,23,8,3,...,922,711,33,23,29,51,12,7,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,21850,20609,2657,3456,3008,1824,1193,485,...,19174,11446,847,478,1023,559,514,404,73,3830
3138,56039,23319,12280,11039,1068,1032,1475,575,569,64,...,13255,7788,1314,555,1098,1056,505,404,97,438
3139,56041,20514,10437,10077,1413,1712,1762,986,341,129,...,8819,5230,606,99,237,230,424,125,62,1806
3140,56043,7768,4004,3764,399,321,606,442,191,76,...,3842,2847,87,128,267,128,22,17,5,341


In [25]:
df = df_acs.copy()

def drop_duplicate_cols(df):
    uniq, idxs = np.unique(df, return_index=True, axis=1)
    return pd.DataFrame(uniq, index=df.index, columns=df.columns[idxs])

# Remove duplicate columns
df = drop_duplicate_cols(df)

## Rename cols
df.rename({'POP_x2':'Pop'},axis=1,inplace=True)
## Reorder columns
df = df[['Fips','Pop'] + [c for c in df.columns if c not in ['Fips','Pop']]]
df.head()

,Fips,Pop,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,EDU_PROF,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,0,0,0,0,0,0,0,1,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,0,6,9,11,12,47,180,14,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,1842,2985,428,499,2642,2860,6892,5415,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,32,90,250,239,9,19,150,102,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,0,0,0,0,0,2,7,1,...,922,369,266,282,411,384,480,443,643,740


### <font color=blue> Get county ratios and insert state pop, state fips, and county ratio cols.

In [26]:
df.insert(2, 'State_fips', 0)
df.insert(3, 'State_pop',0)
df.insert(4, 'Ratio', 0)

df['Fips'] = df.Fips.astype(str)

for tup in df.itertuples():
    
    state_fips = tup.Fips[:-3]
    state_pop = sum(df[df.Fips.str[:-3]==state_fips].Pop)
    ratio = tup.Pop / state_pop
    
    df.at[tup.Index, 'State_fips'] = state_fips
    df.at[tup.Index, 'State_pop'] = state_pop
    df.at[tup.Index, 'Ratio'] = ratio

In [27]:
df

,Fips,Pop,State_fips,State_pop,Ratio,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,2,735951,0.004632,0,0,0,0,0,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,2,735951,0.007135,0,6,9,11,12,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,2,735951,0.397506,1842,2985,428,499,2642,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,2,735951,0.025157,32,90,250,239,9,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,2,735951,0.001154,0,0,0,0,0,...,922,369,266,282,411,384,480,443,643,740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,56,576641,0.073632,165,3135,0,30,73,...,19174,20609,454,506,16349,16838,21850,20884,27816,35626
3138,56039,23319,56,576641,0.040439,209,67,0,74,97,...,13255,11039,30,30,6148,9875,12280,14952,17659,19835
3139,56041,20514,56,576641,0.035575,0,748,0,8,62,...,8819,10077,20,52,8432,7709,10437,9688,13233,18533
3140,56043,7768,56,576641,0.013471,63,106,0,0,5,...,3842,3764,38,71,2992,2958,4004,3907,5423,6651


In [28]:
## SAVE CSV!
DT = timestamp
fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_pop_ratio_{DT}.csv')
df.to_csv(fpath,index=False)
print(DT)

## READ BACK AND CHECK
dff = pd.read_csv(fpath)
display(df, dff)
dff.dtypes

20240928-145111


,Fips,Pop,State_fips,State_pop,Ratio,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,2,735951,0.004632,0,0,0,0,0,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,2,735951,0.007135,0,6,9,11,12,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,2,735951,0.397506,1842,2985,428,499,2642,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,2,735951,0.025157,32,90,250,239,9,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,2,735951,0.001154,0,0,0,0,0,...,922,369,266,282,411,384,480,443,643,740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,56,576641,0.073632,165,3135,0,30,73,...,19174,20609,454,506,16349,16838,21850,20884,27816,35626
3138,56039,23319,56,576641,0.040439,209,67,0,74,97,...,13255,11039,30,30,6148,9875,12280,14952,17659,19835
3139,56041,20514,56,576641,0.035575,0,748,0,8,62,...,8819,10077,20,52,8432,7709,10437,9688,13233,18533
3140,56043,7768,56,576641,0.013471,63,106,0,0,5,...,3842,3764,38,71,2992,2958,4004,3907,5423,6651


,Fips,Pop,State_fips,State_pop,Ratio,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,...,HU_x14,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH
0,2013,3409,2,735951,0.004632,0,0,0,0,0,...,1113,1395,1501,1505,1648,1760,2014,2213,2598,3050
1,2016,5251,2,735951,0.007135,0,6,9,11,12,...,1456,2256,523,549,2626,2471,2995,3655,3838,4740
2,2020,292545,2,735951,0.397506,1842,2985,428,499,2642,...,118055,142897,19827,21150,94213,107273,149648,144598,193312,264762
3,2050,18514,2,735951,0.025157,32,90,250,239,9,...,5992,8790,15427,15640,6485,5770,9724,6454,10025,18131
4,2060,849,2,735951,0.001154,0,0,0,0,0,...,922,369,266,282,411,384,480,443,643,740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,42459,56,576641,0.073632,165,3135,0,30,73,...,19174,20609,454,506,16349,16838,21850,20884,27816,35626
3138,56039,23319,56,576641,0.040439,209,67,0,74,97,...,13255,11039,30,30,6148,9875,12280,14952,17659,19835
3139,56041,20514,56,576641,0.035575,0,748,0,8,62,...,8819,10077,20,52,8432,7709,10437,9688,13233,18533
3140,56043,7768,56,576641,0.013471,63,106,0,0,5,...,3842,3764,38,71,2992,2958,4004,3907,5423,6651


Fips                      int64
Pop                       int64
State_fips                int64
State_pop                 int64
Ratio                   float64
OCC_MBSA_ELCAM_L          int64
IND_AFFHM_MQE             int64
IND_PSMAW_M               int64
HU_OCC_OPRgt200           int64
HU_UIS50P                 int64
IND_WHT_x7                int64
HU_UIS2049                int64
EDU_PROF                  int64
HH_7pP                    int64
OCC_MBSA_CES_CM           int64
OCC_MBSA_HCPT_HTT         int64
OCC_MBSA_ELCAM_ADESM      int64
HU_UIS01A                 int64
IND_FIRR_FI               int64
OCC_SERV_HCS_x11          int64
OCC_SERV_PS_LE            int64
IND_INF_x7                int64
EDU_DOCT                  int64
POP_NHPINH                int64
POP_NHPI                  int64
OCC_MBSA_ELCAM_CSS        int64
IND_PSMAW_PST             int64
HHI_35_40k                int64
HU_OCC_OPR151200          int64
HHI_25_30k                int64
HU_OCC_OPR101150          int64
OCC_SERV